In [1]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))


In [19]:
import requests
import json

# 요청 URL
url = "https://w1bghm6ujn-3.algolianet.com/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20JavaScript%20(3.35.1)%3B%20Browser%3B%20instantsearch.js%20(4.62.0)%3B%20Vue%20(2.7.15)%3B%20Vue%20InstantSearch%20(4.13.1)%3B%20JS%20Helper%20(3.16.0)&x-algolia-application-id=W1BGHM6UJN&x-algolia-api-key=8b54f30c2c74c6930f51d8d40ce06b3a"

# 요청 헤더
headers = {
    "accept": "application/json",
    "accept-language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "content-type": "application/x-www-form-urlencoded",
    "sec-ch-ua": "\"Not_A Brand\";v=\"8\", \"Chromium\";v=\"120\", \"Google Chrome\";v=\"120\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "cross-site",
    "Referer": "https://en.salon-agriculture.com/",
    "Referrer-Policy": "strict-origin-when-cross-origin"
}


# 0 - 22
def api_call(page):
    # 요청 본문
    data = {
        "requests": [
            {
                "indexName": "catalog.prod.salon_agriculture.exhibitors.en",
                "params": f"facets=%5B%22address.country%22%2C%22brands.name%22%2C%22businessArea.categories.lvl0%22%2C%22stands.hall%22%2C%22stands.sector%22%2C%22targets%22%2C%22thematics%22%5D&highlightPostTag=__%2Fais-highlight__&highlightPreTag=__ais-highlight__&maxValuesPerFacet=100&page={page}&query=&tagFilters="
            }
        ]
    }

    # next_data = {
    #     "requests":[
    #         {
    #             "indexName":"catalog.prod.salon_agriculture.exhibitors.en",
    #             "params":"facets=%5B%22address.country%22%2C%22brands.name%22%2C%22businessArea.categories.lvl0%22%2C%22stands.hall%22%2C%22stands.sector%22%2C%22targets%22%2C%22thematics%22%5D&highlightPostTag=__%2Fais-highlight__&highlightPreTag=__ais-highlight__&maxValuesPerFacet=100&page=1&query=&tagFilters="
    #         }
    #     ]
    # }

    # POST 요청 수행
    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        # 응답 출력
        #print(response.text)
        j_data = response.json()
    
        return j_data
    else:
        print('  ** error api call :', response)
        return None
    


In [20]:
address_parts = ['address_1', 'address_2', 'address_3', 'city', 'state', 'zipcode', 'country']
def get_packer_data(j_data):
    result_list = []
    for item in j_data['results'][0]['hits']:
        result = {}

        result['name'] = item['name']
        result['address'] = ', '.join([item['address'].get(sub_item, '') for sub_item in address_parts if item['address'].get(sub_item)])
        result['businessArea'] = '; '.join([sub_item['name'] for sub_item in item.get('businessArea', [])])
        result['website'] = item.get('urls', {}).get('website', '')

        result_list.append(result)
    return result_list



In [22]:
result_list = []
for i in range(0, 23):
    j_data = api_call(i)
    packer_list = get_packer_data(j_data)

    result_list.append(packer_list)

In [26]:
total = sum((len(result) for result in result_list))

In [31]:
new_result_list = []
for result in result_list:
    new_result_list.extend(result)

import pandas as pd

df = pd.DataFrame(new_result_list)


In [33]:
df.to_csv('./output/salon_agriculture_20240205.csv', index=False, quoting=1)